In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# Create the dataframe
# data = {
#    'Feature1': [1.5, 2.0, 1.8, np.nan, 2.2],
#    'Feature2': [3.2, 2.1, 2.5, 2.9, 3.6],
#    'Feature3': [5.1, 4.9, 5.3, 4.7, 5.0],
#    'Feature4': [6.5, 6.7, np.nan, 6.4, 6.1],
#    'Category': [1, 2, 3, 2, 1]
# }

# data = pd.DataFrame(data)

import pandas as pd
import numpy as np

# Create the dataframe
data = {
    'Feature1': [1.5, 2.0, 1.8, np.nan, 2.2, 3.7, 4.1, 4.3, 3.3, 3.9],
    'Feature2': [3.2, 2.5, np.nan, 2.9, 3.6, 4.2, 4.8, 4.4, 4.7, 4.6],
    'Feature3': [5.1, 4.9, 5.3, 4.7, 5.0, 5.2, 6.2, 6.9, 5.8, 6.0],
    'Feature4': [6.5, 6.7, np.nan, 6.4, 6.1, 7.0, 6.1, 7.2, 7.5, 6.9],
    'Category': [1, 2, 3, 2, 1, 2, 3, 1, 2, 3]
}

data = pd.DataFrame(data)

# Display the dataframe
data

,Feature1,Feature2,Feature3,Feature4,Category
0,1.5,3.2,5.1,6.5,1
1,2.0,2.5,4.9,6.7,2
2,1.8,NaN,5.3,NaN,3
3,NaN,2.9,4.7,6.4,2
4,2.2,3.6,5.0,6.1,1
5,3.7,4.2,5.2,7.0,2
6,4.1,4.8,6.2,6.1,3
7,4.3,4.4,6.9,7.2,1
8,3.3,4.7,5.8,7.5,2
9,3.9,4.6,6.0,6.9,3


In [7]:
def filter_missing(df):
    df = df.dropna(axis=0)
    return df

def collect_missing(df):
    df = df[df.isna().any(axis=1)]
    return df

def determine_missing_proportions(df):
    f = []; p = []
    for col in df.columns:
        f.append(col)
        p.append(df[col].isna().sum()/len(df))
    d = dict(zip(f, p))
    return(d)

In [8]:
nm = filter_missing(data)
nm

,Feature1,Feature2,Feature3,Feature4,Category
0,1.5,3.2,5.1,6.5,1
1,2.0,2.5,4.9,6.7,2
4,2.2,3.6,5.0,6.1,1
5,3.7,4.2,5.2,7.0,2
6,4.1,4.8,6.2,6.1,3
7,4.3,4.4,6.9,7.2,1
8,3.3,4.7,5.8,7.5,2
9,3.9,4.6,6.0,6.9,3


In [9]:
cm = collect_missing(data)
cm

,Feature1,Feature2,Feature3,Feature4,Category
2,1.8,NaN,5.3,NaN,3
3,NaN,2.9,4.7,6.4,2


In [10]:
res = determine_missing_proportions(data)
res

{'Feature1': 0.1,
 'Feature2': 0.1,
 'Feature3': 0.0,
 'Feature4': 0.1,
 'Category': 0.0}

In [29]:
for ind in cm.index:
    print(np.array(cm.loc[[ind]].dropna(axis=1)))

[[1.8 5.3 3. ]]
[[2.9 4.7 6.4 2. ]]


In [32]:
"""
!pip install wget 

import wget
wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns
"""

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ae26a6b0aa33e71b78d68f019f854ccba5e42aa178b951eeb72b98fd32d034a6
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [33]:
"""
# Function produce_NA for generating missing values ------------------------------------------------------

def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    #\"""
    from: https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    #\"""
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}
    """

In [34]:
"""
# Create the dataframe
data = {
    'Feature1': [1.5, 2.0, 1.8, 2.0, 2.2],
    'Feature2': [3.2, 2.1, 2.5, 2.9, 3.6],
    'Feature3': [5.1, 4.9, 5.3, 4.7, 5.0],
    'Feature4': [6.5, 6.7, 7.3, 6.4, 6.1],
    'Category': [1, 2, 3, 2, 1]
}

data = pd.DataFrame(data)
"""

In [4]:
from sklearn.neighbors import NearestNeighbors

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def simulate_missing(df, method):
    def filter_missing(df):
        df = df.dropna(axis=0)
        return df

    def collect_missing(df):
        df = df[df.isna().any(axis=1)]
        return df

    def determine_missing_proportions(df):
        f = []; p = []
        for col in df.columns:
            f.append(col)
            p.append(df[col].isna().sum()/len(df))
        d = dict(zip(f, p))
        return(d)
    
    nm = filter_missing(df)
    cm = collect_missing(df)
    mp = determine_missing_proportions(df)
    
    if method == "nearest-neighbors":
        nbrs = NearestNeighbors(n_neighbors=1).fit(nm)